# Algoritmo Genético

### Exercício 1

In [ ]:
import random
from datetime import datetime


def funcao_fitness(tentativa):
    return sum(1 for esperado, atual in zip(target, tentativa)
               if esperado == atual)


def mostrar_resultado(tentativa):
    deltaTempo = datetime.now() - inicioTempo
    fitness = funcao_fitness(tentativa)
    print("{0}\t{1}\t{2}".format(tentativa, fitness, str(deltaTempo)))


def criar_cromossomo(tamanho):
    genes = []
    while len(genes) < tamanho:
        sampleSize = min(tamanho - len(genes), len(geneSet))
        genes.extend(random.sample(geneSet, sampleSize))
    return ''.join(genes)


def mutacao(cromossomo):
    index = random.randrange(0, len(cromossomo))
    cromossomo_filho = list(cromossomo)
    novoGene, alternar = random.sample(geneSet, 2)
    cromossomo_filho[index] = alternar if novoGene == cromossomo_filho[index] else novoGene
    return ''.join(cromossomo_filho)


random.seed()
geneSet = " abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!."
target = "Hello World!"
inicioTempo = datetime.now()
melhor_cromossomo = criar_cromossomo(len(target))
melhor_fitness = funcao_fitness(melhor_cromossomo)
mostrar_resultado(melhor_cromossomo)
while True:
    filho = mutacao(melhor_cromossomo)
    filho_fitness = funcao_fitness(filho)

    if melhor_fitness >= filho_fitness:
        continue
    mostrar_resultado(filho)
    if filho_fitness >= len(melhor_cromossomo):
        break
    melhor_fitness = filho_fitness
    melhor_cromossomo = filho

Outra forma de resolver o mesmo problema:

Fonte: [Github](https://gist.github.com/desertmonad/2299154)

In [ ]:
from collections import namedtuple
from operator import itemgetter
import string
import random


GA_POPSIZE = 1024    #ga population size
GA_MAXITER = 200     #maximum iterations
GA_ELITRATE = 0.2      #elitism rate (varname [sic])
GA_MUTATIONRATE = 0.25  
GA_MUTATION = 32768 * GA_MUTATIONRATE
GA_TARGET = "Hello World"

ga_struct = namedtuple("ga_struct", [ "string", "fitness"])


def init_population(ga_population, ga_buffer):
    tsize = len(GA_TARGET)

    for i in range(0,GA_POPSIZE):
        random_string = "".join((random.choice(string.ascii_letters+" ") for x in range(0,tsize)))
        citizen = ga_struct(random_string,0)
        
        ga_population.append(citizen)
        ga_buffer.append(ga_struct(" ",0))

def calc_fitness(ga_population):
    pop_index = 0
    for item in ga_population:
        index = 0
        fitness = 0
        for letter in item.string:
            fitness += abs(ord(letter) - ord(GA_TARGET[index]))
            index += 1
        ga_population[pop_index]  =  item._replace(fitness=fitness)
        pop_index += 1


def sort_by_fitness(ga_population):
    ga_population.sort(key=itemgetter(1))

def mutate(ga_population, index):
    rand_handler = random.Random()
    tsize = len(GA_TARGET)
    ipos = rand_handler.randint(0,tsize-1) - 1
    new_letter = random.choice(string.ascii_letters)
    citizen = ga_population[index]

    temp_list = list(citizen.string)
    temp_list[ipos]=new_letter
    citizen = citizen._replace(string="".join(temp_list))

    ga_population[index] = citizen


def elitism(ga_population, ga_buffer, esize):
    for i in range(0, esize):
        ga_buffer[i] = ga_population[i]
        
def mate(ga_population, ga_buffer):
    esize = int(GA_POPSIZE * GA_ELITRATE)
    tsize = len(GA_TARGET)
    rand_handler = random.Random()

    elitism(ga_population, ga_buffer, esize)

    #mate the rest
    for i in range(esize,GA_POPSIZE):
        i1 = rand_handler.randint(0,int(GA_POPSIZE/2))
        i2 = rand_handler.randint(0,int(GA_POPSIZE/2))
        spos = rand_handler.randint(0,tsize)
        
        ga_buffer[i] = ga_struct(ga_population[i1].string[0:spos] + ga_population[i2].string[spos:tsize],ga_buffer[i].fitness)
        if rand_handler.random() < GA_MUTATION:
           mutate(ga_buffer,i)
            
def main():
    my_population = []
    my_buffer = []

    init_population(my_population,my_buffer)

    last = 0
    for i in range(0,GA_MAXITER):
        calc_fitness(my_population)        
        sort_by_fitness(my_population)

        if my_population[0].fitness != last:
            print(" iter: %d,\t%s, \t score:%d " % (i, my_population[0].string, my_population[0].fitness))
            last = my_population[0].fitness

        if my_population[0].fitness == 0:
            break
        
        mate(my_population, my_buffer)
        (my_buffer, my_population) = (my_population, my_buffer) #swap

    print( "done")

if __name__=="__main__":
    main()